In [ ]:
POP_SIZE = 500
MUTATION_RATE = 0.4
CROSSOVER_RATE_PARENT = 0.8
TOURNAMENT_SIZE = 3
ELITISM_SIZE = 5
MAX_GENERATIONS = 50000

In [ ]:
import copy  # Wajib import ini buat Elitism yang aman
import time
import numpy as np
import random
import itertools

easy1_sudoku = np.array([
    [0, 4, 0, 0, 3, 8, 0, 0, 7],
    [0, 7, 9, 0, 1, 0, 0, 0, 8],
    [0, 5, 8, 0, 0, 0, 0, 2, 0],
    [0, 6, 4, 0, 0, 5, 0, 0, 3],
    [0, 0, 7, 3, 0, 4, 8, 0, 0],
    [2, 0, 0, 7, 0, 0, 5, 6, 0],
    [0, 2, 0, 0, 0, 0, 7, 3, 0],
    [7, 0, 0, 0, 5, 0, 6, 4, 0],
    [4, 0, 0, 1, 9, 0, 0, 8, 5]
])

medium1_sudoku = np.array([
    [6, 0, 0, 0, 3, 0, 0, 0, 1],
    [0, 1, 0, 6, 9, 0, 2, 8, 0],
    [5, 0, 9, 0, 0, 0, 0, 0, 0],
    [0, 6, 2, 0, 8, 3, 0, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 0, 2, 7, 0, 3, 1, 0],
    [0, 0, 0, 0, 0, 0, 5, 0, 2],
    [0, 5, 4, 0, 6, 7, 0, 9, 0],
    [9, 0, 0, 0, 5, 0, 0, 0, 8]
])

hard1_sudoku = np.array([
    [9, 5, 6, 3, 0, 1, 8, 0, 0],
    [0, 0, 0, 0, 4, 0, 2, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 9],
    [0, 6, 0, 4, 0, 0, 5, 0, 0],
    [4, 0, 0, 0, 6, 0, 0, 0, 7],
    [0, 0, 1, 0, 0, 2, 0, 6, 0],
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 9, 0, 0, 0, 0],
    [0, 0, 2, 7, 0, 4, 9, 3, 6],
])

# ------------------------------
# Pretty print Sudoku grid
# ------------------------------
def print_sudoku(grid, title=None):
    if title:
        print("\n" + "=" * 40)
        print(title)
        print("=" * 40)
    for i, row in enumerate(grid):
        row_str = ""
        for j, val in enumerate(row):
            char = "." if val == 0 else str(val)
            sep = " "
            if j in [2, 5]:
                sep = " | "
            row_str += char + sep
        print(row_str)
        if i in [2, 5]:
            print("-" * 21)
    print("=" * 40)

# ------------------------------
# Precompute candidate map (dari GIVENS saja)
# ------------------------------
def precompute_candidate_map(base_grid):
    fixed = base_grid != 0
    cand = {}
    for r in range(9):
        row_used = set(base_grid[r, fixed[r, :]])
        for c in range(9):
            if fixed[r, c]:
                continue
            col_used = set(base_grid[fixed[:, c], c])
            br, bc = (r // 3) * 3, (c // 3) * 3
            block = base_grid[br:br+3, bc:bc+3]
            block_fixed = fixed[br:br+3, bc:bc+3]
            block_used = set(block[block_fixed])
            used = row_used | col_used | block_used
            cand[(r, c)] = set(range(1, 10)) - used
    return cand, fixed

# ------------------------------
# Posisi non-given per blok 3x3
# ------------------------------
def compute_block_positions(base_grid, fixed):
    block_pos = {}
    for bx in range(3):
        for by in range(3):
            rows = range(bx*3, (bx+1)*3)
            cols = range(by*3, (by+1)*3)
            positions = []
            for r in rows:
                for c in cols:
                    if not fixed[r, c]:
                        positions.append((r, c))
            block_pos[(bx, by)] = positions
    return block_pos

# ------------------------------
# Individual: hanya angka non-given per blok
# ------------------------------
def create_individual(base_grid, fixed, block_pos):
    ind = {}
    for (bx, by), positions in block_pos.items():
        br, bc = bx*3, by*3
        block = base_grid[br:br+3, bc:bc+3]
        block_fixed_mask = fixed[br:br+3, bc:bc+3]
        used = set(block[block_fixed_mask])
        missing = list(set(range(1, 10)) - used)         # pastikan blok valid (1..9)
        random.shuffle(missing)
        # len(missing) == jumlah sel non-given pada blok
        ind[(bx, by)] = missing[:len(positions)]
    return ind

# ------------------------------
# Decode: isi non-given ke grid copy
# ------------------------------
def decode_to_grid(base_grid, individual, block_pos):
    grid = base_grid.copy()
    for (bx, by), positions in block_pos.items():
        vals = individual[(bx, by)]
        for i, (r, c) in enumerate(positions):
            grid[r, c] = vals[i]
    return grid

# ------------------------------
# Fitness: duplikasi baris + kolom
# (blok valid by construction)
# ------------------------------
def fitness_individual(individual, base_grid, block_pos):
    # buat set berisi nilai unik per baris dan kolom
    row_vals = [set() for _ in range(9)]
    col_vals = [set() for _ in range(9)]

    # tambahkan semua angka 'given' dulu
    fixed = base_grid != 0
    for r in range(9):
        for c in range(9):
            if fixed[r, c]:
                val = base_grid[r, c]
                row_vals[r].add(val)
                col_vals[c].add(val)

    # tambahkan semua angka dari individu
    for (bx, by), positions in block_pos.items():
        vals = individual[(bx, by)]
        for i, (r, c) in enumerate(positions):
            v = vals[i]
            row_vals[r].add(v)
            col_vals[c].add(v)

    # hitung penalti duplikasi
    score = 0
    for i in range(9):
        score += (9 - len(row_vals[i]))  # baris
        score += (9 - len(col_vals[i]))  # kolom

    return score


# ------------------------------
# Tournament selection
# ------------------------------
def tournament_selection(population, base_grid, block_pos, k=TOURNAMENT_SIZE):
    selected = random.sample(population, k)
    selected.sort(key=lambda ind: fitness_individual(ind, base_grid, block_pos))
    return selected[0]

# ------------------------------
# Crossover: block-wise
# ------------------------------

def crossover1(p1, p2):
    child = {}
    for bx in range(3):
        for by in range(3):
            key = (bx, by)
            # ambil blok dari salah satu parent
            child[key] = list(p1[key]) if random.random() < 0.5 else list(p2[key])
    return child

def crossover2(p1, p2, base_grid, block_pos):
    """
    Versi efisien tanpa decode_to_grid.
    Menilai skor baris dan kolom langsung dari genotipe per-blok.
    """
    # precompute: isi baris dan kolom untuk kedua parent
    fixed = base_grid != 0
    row_vals_p1 = [set(base_grid[r, fixed[r, :]]) for r in range(9)]
    col_vals_p1 = [set(base_grid[fixed[:, c], c]) for c in range(9)]
    row_vals_p2 = [set(rv) for rv in row_vals_p1]
    col_vals_p2 = [set(cv) for cv in col_vals_p1]

    for (bx, by), positions in block_pos.items():
        vals1 = p1[(bx, by)]
        vals2 = p2[(bx, by)]
        for i, (r, c) in enumerate(positions):
            row_vals_p1[r].add(vals1[i])
            col_vals_p1[c].add(vals1[i])
            row_vals_p2[r].add(vals2[i])
            col_vals_p2[c].add(vals2[i])

    def row_score(row_vals):  # total unique per 3 baris
        return sum(len(row_vals[r]) for r in rows)

    def col_score(col_vals):
        return sum(len(col_vals[c]) for c in cols)

    # hasil child berdasarkan row dan col
    child_row, child_col = {}, {}

    for bx in range(3):
        for by in range(3):
            rows = range(bx*3, (bx+1)*3)
            cols = range(by*3, (by+1)*3)

            sum_r_p1 = sum(len(row_vals_p1[r]) for r in rows)
            sum_r_p2 = sum(len(row_vals_p2[r]) for r in rows)
            sum_c_p1 = sum(len(col_vals_p1[c]) for c in cols)
            sum_c_p2 = sum(len(col_vals_p2[c]) for c in cols)

            child_row[(bx, by)] = list(p2[(bx, by)]) if sum_r_p2 > sum_r_p1 else list(p1[(bx, by)])
            child_col[(bx, by)] = list(p2[(bx, by)]) if sum_c_p2 > sum_c_p1 else list(p1[(bx, by)])

    fit_r = fitness_individual(child_row, base_grid, block_pos)
    fit_c = fitness_individual(child_col, base_grid, block_pos)
    return child_row if fit_r <= fit_c else child_col

# ------------------------------
# MUTATE
# ------------------------------
def mutate(individual, mutation_rate, candidate_map, block_pos):
    new_ind = {k: list(v) for k, v in individual.items()}

    for (bx, by), positions in block_pos.items():
        if len(positions) < 2:
            continue

        # Cuma mutasi kalau random < rate
        if random.random() >= mutation_rate:
            continue

        pairs = list(itertools.combinations(range(len(positions)), 2))
        random.shuffle(pairs)

        for i, j in pairs:
            (r1, c1) = positions[i]
            (r2, c2) = positions[j]
            v1 = new_ind[(bx, by)][i]
            v2 = new_ind[(bx, by)][j]

            # Cek Candidate Map
            cand1 = candidate_map.get((r1, c1), set())
            cand2 = candidate_map.get((r2, c2), set())

            if (v1 in cand2) or (v2 in cand1):
                new_ind[(bx, by)][i], new_ind[(bx, by)][j] = v2, v1
                break

    return new_ind

# ==========================================
# LOCAL SEARCH
# ==========================================
def try_local_fix(individual, base_grid, block_pos):
    best_ind = individual
    # Hitung fitness awal
    current_best_fit = fitness_individual(individual, base_grid, block_pos)

    # Loop semua blok
    for (bx, by), positions in block_pos.items():
        if len(positions) < 2: continue

        # Coba semua kemungkinan tukeran di blok ini
        pairs = list(itertools.combinations(range(len(positions)), 2))

        # Kita clone dulu biar gak ngerusak yang asli
        temp_ind = {k: list(v) for k, v in individual.items()}

        for i, j in pairs:
            # Lakukan Swap
            temp_ind[(bx, by)][i], temp_ind[(bx, by)][j] = temp_ind[(bx, by)][j], temp_ind[(bx, by)][i]

            # Cek skor barunya
            fit = fitness_individual(temp_ind, base_grid, block_pos)

            if fit < current_best_fit:
                return temp_ind, fit

            # Kalau gak bagus, balikin lagi (Swap back) biar bisa cek pasangan lain
            temp_ind[(bx, by)][i], temp_ind[(bx, by)][j] = temp_ind[(bx, by)][j], temp_ind[(bx, by)][i]

    return best_ind, current_best_fit

# ==========================================
# FUNGSI SOLVER
# ==========================================
def genetic_sudoku_solver(base_grid, crossover_func):
    candidate_map, fixed = precompute_candidate_map(base_grid)
    block_pos = compute_block_positions(base_grid, fixed)
    population = [create_individual(base_grid, fixed, block_pos) for _ in range(POP_SIZE)]

    print(f"\nMulai evolusi Hybrid (GA + Local Search)...")
    total_start = time.time()

    # Variabel Restart
    stuck_count = 0
    last_best_fit = float('inf')
    RESTART_LIMIT = 1000 # Restart kalau stuck 1000 gen

    for gen in range(MAX_GENERATIONS):
        gen_start = time.time()

        # 1. Evaluasi & Sort
        population.sort(key=lambda ind: fitness_individual(ind, base_grid, block_pos))
        best = population[0]
        best_fit = fitness_individual(best, base_grid, block_pos)

        # (LOCAL SEARCH)
        # Kalau errornya tinggal dikit (<= 4), kita suruh komputer cari manual!
        if best_fit > 0 and best_fit <= 4:
            refined_ind, refined_fit = try_local_fix(best, base_grid, block_pos)

            if refined_fit < best_fit:
                print(f"Local Search! {best_fit} -> {refined_fit}")
                best = refined_ind
                best_fit = refined_fit
                population[0] = best

        # Logika Restart
        if best_fit == last_best_fit:
            stuck_count += 1
        else:
            stuck_count = 0
            last_best_fit = best_fit

        if stuck_count > RESTART_LIMIT:
            print(f"\n STUCK {best_fit} selama {RESTART_LIMIT} gen! RESTART...")
            population = [create_individual(base_grid, fixed, block_pos) for _ in range(POP_SIZE)]
            stuck_count = 0
            last_best_fit = float('inf')
            continue

        # Logging
        if gen % 100 == 0:
            gen_time = time.time() - gen_start
            print(f"Gen {gen:4d} | Fitness terbaik = {best_fit:2d} | Stuck: {stuck_count}")

        # Cek Solusi
        if best_fit == 0:
            total_time = time.time() - total_start
            print(f"\n✅ Solusi ditemukan di generasi {gen}!")
            print(f"Total waktu: {total_time:.2f} detik")
            return decode_to_grid(base_grid, best, block_pos)

        # 2. Buat Populasi Baru
        new_population = []

        # Elitism
        elites = copy.deepcopy(population[:ELITISM_SIZE])
        new_population.extend(elites)

        # Offspring
        num_offspring = POP_SIZE - ELITISM_SIZE
        for _ in range(num_offspring):
            p1 = tournament_selection(population, base_grid, block_pos)
            p2 = tournament_selection(population, base_grid, block_pos)

            if random.random() < CROSSOVER_RATE_PARENT:
                child = crossover_func(p1, p2)
            else:
                child = p1 if fitness_individual(p1, base_grid, block_pos) < fitness_individual(p2, base_grid, block_pos) else p2
                child = {k: list(v) for k, v in child.items()}

            child = mutate(child, MUTATION_RATE, candidate_map, block_pos)
            new_population.append(child)

        population = new_population

    print("\n❌ Tidak ditemukan solusi.")
    return decode_to_grid(base_grid, best, block_pos)

In [ ]:
def run_experiment(seed, crossover_func, sudoku_grid, label=""):
    random.seed(seed)
    np.random.seed(seed)

    print(f"\n=== Eksperimen Seed {seed} | Puzzle {label or 'Custom'} | "
          f"Crossover = {crossover_func.__name__} ===")

    solution = genetic_sudoku_solver(sudoku_grid, crossover_func=crossover_func)
    print_sudoku(solution, f"Solusi Akhir (Seed {seed} - {label or 'Custom'})")

In [ ]:
SEED = 74

C1

In [ ]:
run_experiment(SEED, crossover1, hard1_sudoku)


=== Eksperimen Seed 74 | Puzzle Custom | Crossover = crossover1 ===

Mulai evolusi Hybrid (GA + Local Search)...
Gen    0 | Fitness terbaik = 30 | Stuck: 0
Gen  100 | Fitness terbaik = 10 | Stuck: 61
Gen  200 | Fitness terbaik =  6 | Stuck: 1
Gen  300 | Fitness terbaik =  5 | Stuck: 81
Local Search! 4 -> 3
Gen  400 | Fitness terbaik =  3 | Stuck: 39
Gen  500 | Fitness terbaik =  3 | Stuck: 139
Gen  600 | Fitness terbaik =  3 | Stuck: 239
Gen  700 | Fitness terbaik =  3 | Stuck: 339
Gen  800 | Fitness terbaik =  3 | Stuck: 439
Gen  900 | Fitness terbaik =  3 | Stuck: 539
Gen 1000 | Fitness terbaik =  3 | Stuck: 639
Gen 1100 | Fitness terbaik =  3 | Stuck: 739
Gen 1200 | Fitness terbaik =  3 | Stuck: 839
Gen 1300 | Fitness terbaik =  3 | Stuck: 939

 STUCK 3 selama 1000 gen! RESTART...
Gen 1400 | Fitness terbaik = 13 | Stuck: 5
Gen 1500 | Fitness terbaik =  7 | Stuck: 22
Gen 1600 | Fitness terbaik =  7 | Stuck: 122
Gen 1700 | Fitness terbaik =  6 | Stuck: 25
Gen 1800 | Fitness terbaik =

In [ ]:
run_experiment(SEED, crossover1, medium1_sudoku)


=== Eksperimen Seed 74 | Puzzle Custom | Crossover = crossover1 ===

Mulai evolusi Hybrid (GA + Local Search)...
Gen    0 | Fitness terbaik = 31 | Stuck: 0
Gen  100 | Fitness terbaik =  8 | Stuck: 54
Gen  200 | Fitness terbaik =  6 | Stuck: 44
Gen  300 | Fitness terbaik =  6 | Stuck: 144
Gen  400 | Fitness terbaik =  6 | Stuck: 244
Gen  500 | Fitness terbaik =  6 | Stuck: 344
Gen  600 | Fitness terbaik =  6 | Stuck: 444
Gen  700 | Fitness terbaik =  4 | Stuck: 64
Gen  800 | Fitness terbaik =  4 | Stuck: 164
Gen  900 | Fitness terbaik =  4 | Stuck: 264
Gen 1000 | Fitness terbaik =  4 | Stuck: 364
Gen 1100 | Fitness terbaik =  4 | Stuck: 464
Gen 1200 | Fitness terbaik =  4 | Stuck: 564
Gen 1300 | Fitness terbaik =  4 | Stuck: 664
Gen 1400 | Fitness terbaik =  4 | Stuck: 764
Gen 1500 | Fitness terbaik =  4 | Stuck: 864
Gen 1600 | Fitness terbaik =  4 | Stuck: 964

 STUCK 4 selama 1000 gen! RESTART...
Gen 1700 | Fitness terbaik =  7 | Stuck: 5
Gen 1800 | Fitness terbaik =  7 | Stuck: 105


In [ ]:
run_experiment(SEED, crossover1, easy1_sudoku)


=== Eksperimen Seed 74 | Puzzle Custom | Crossover = crossover1 ===

Mulai evolusi Hybrid (GA + Local Search)...
Gen    0 | Fitness terbaik = 30 | Stuck: 0
Gen  100 | Fitness terbaik =  5 | Stuck: 19
Local Search! 4 -> 2
Gen  200 | Fitness terbaik =  2 | Stuck: 97
Gen  300 | Fitness terbaik =  2 | Stuck: 197
Gen  400 | Fitness terbaik =  2 | Stuck: 297
Gen  500 | Fitness terbaik =  2 | Stuck: 397
Gen  600 | Fitness terbaik =  2 | Stuck: 497
Gen  700 | Fitness terbaik =  2 | Stuck: 597
Gen  800 | Fitness terbaik =  2 | Stuck: 697
Gen  900 | Fitness terbaik =  2 | Stuck: 797
Gen 1000 | Fitness terbaik =  2 | Stuck: 897
Gen 1100 | Fitness terbaik =  2 | Stuck: 997

 STUCK 2 selama 1000 gen! RESTART...
Gen 1200 | Fitness terbaik =  6 | Stuck: 56
Gen 1300 | Fitness terbaik =  5 | Stuck: 19
Gen 1400 | Fitness terbaik =  4 | Stuck: 12
Gen 1500 | Fitness terbaik =  4 | Stuck: 112
Gen 1600 | Fitness terbaik =  4 | Stuck: 212
Gen 1700 | Fitness terbaik =  4 | Stuck: 312
Gen 1800 | Fitness terba

C2

In [ ]:
block_pos_hard = compute_block_positions(hard1_sudoku, hard1_sudoku != 0)

run_experiment(
    SEED,
    lambda p1, p2: crossover2(p1, p2, hard1_sudoku, block_pos_hard),
    hard1_sudoku
)


=== Eksperimen Seed 74 | Puzzle Custom | Crossover = <lambda> ===

Mulai evolusi Hybrid (GA + Local Search)...
Gen    0 | Fitness terbaik = 30 | Stuck: 0
Local Search! 4 -> 2
Gen  100 | Fitness terbaik =  2 | Stuck: 73
Gen  200 | Fitness terbaik =  2 | Stuck: 173
Gen  300 | Fitness terbaik =  2 | Stuck: 273
Gen  400 | Fitness terbaik =  2 | Stuck: 373
Gen  500 | Fitness terbaik =  2 | Stuck: 473
Gen  600 | Fitness terbaik =  2 | Stuck: 573
Gen  700 | Fitness terbaik =  2 | Stuck: 673
Gen  800 | Fitness terbaik =  2 | Stuck: 773
Gen  900 | Fitness terbaik =  2 | Stuck: 873
Gen 1000 | Fitness terbaik =  2 | Stuck: 973

 STUCK 2 selama 1000 gen! RESTART...
Gen 1100 | Fitness terbaik =  2 | Stuck: 11
Gen 1200 | Fitness terbaik =  2 | Stuck: 111
Gen 1300 | Fitness terbaik =  2 | Stuck: 211
Gen 1400 | Fitness terbaik =  2 | Stuck: 311
Gen 1500 | Fitness terbaik =  2 | Stuck: 411
Gen 1600 | Fitness terbaik =  2 | Stuck: 511
Gen 1700 | Fitness terbaik =  2 | Stuck: 611
Gen 1800 | Fitness terb

In [ ]:
block_pos_medium = compute_block_positions(medium1_sudoku, medium1_sudoku != 0)

run_experiment(
    SEED,
    lambda p1, p2: crossover2(p1, p2, medium1_sudoku, block_pos_medium),
    medium1_sudoku
)


=== Eksperimen Seed 74 | Puzzle Custom | Crossover = <lambda> ===

Mulai evolusi Hybrid (GA + Local Search)...
Gen    0 | Fitness terbaik = 31 | Stuck: 0
Local Search! 4 -> 2
Local Search! 2 -> 0

✅ Solusi ditemukan di generasi 27!
Total waktu: 7.78 detik

Solusi Akhir (Seed 74 - Custom)
6 2 8 | 7 3 5 | 9 4 1 
3 1 7 | 6 9 4 | 2 8 5 
5 4 9 | 1 2 8 | 6 3 7 
---------------------
1 6 2 | 4 8 3 | 7 5 9 
7 9 3 | 5 1 6 | 8 2 4 
4 8 5 | 2 7 9 | 3 1 6 
---------------------
8 3 6 | 9 4 1 | 5 7 2 
2 5 4 | 8 6 7 | 1 9 3 
9 7 1 | 3 5 2 | 4 6 8 


In [ ]:
block_pos_easy = compute_block_positions(easy1_sudoku, easy1_sudoku != 0)

run_experiment(
    SEED,
    lambda p1, p2: crossover2(p1, p2, easy1_sudoku, block_pos_easy),
    easy1_sudoku
)


=== Eksperimen Seed 74 | Puzzle Custom | Crossover = <lambda> ===

Mulai evolusi Hybrid (GA + Local Search)...
Gen    0 | Fitness terbaik = 30 | Stuck: 0
Local Search! 3 -> 2

✅ Solusi ditemukan di generasi 62!
Total waktu: 17.54 detik

Solusi Akhir (Seed 74 - Custom)
6 4 2 | 5 3 8 | 9 1 7 
3 7 9 | 6 1 2 | 4 5 8 
1 5 8 | 4 7 9 | 3 2 6 
---------------------
8 6 4 | 9 2 5 | 1 7 3 
5 1 7 | 3 6 4 | 8 9 2 
2 9 3 | 7 8 1 | 5 6 4 
---------------------
9 2 5 | 8 4 6 | 7 3 1 
7 8 1 | 2 5 3 | 6 4 9 
4 3 6 | 1 9 7 | 2 8 5 
